In [1]:
# repo url(https://github.com/xwang71785/buffett.git)
# 只针对中国大陆股票市场
# 仅限于概念验证，勿用于实际投资中
# QQ：2908683294
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
import pandas_datareader.data as web
import datetime as dt
import matplotlib.dates as mdates

#### 1.估值
实体现金流量净现值估值模型



In [3]:
# 基于网易构造三张财务报表的数据源URL
server = "http://quotes.money.163.com/"
path = "service/"
table1 = "zcfzb_"
table2 = "lrb_"
table3 = "xjllb_"
stock = "600585"    # 根据需求变更stock中的股票代码
type = ".html?type=year"

target1 = server + path + table1 + stock + type
target2 = server + path + table2 + stock + type
target3 = server + path + table3 + stock + type

print(target1, '\n', target2, '\n', target3)

http://quotes.money.163.com/service/zcfzb_600585.html?type=year 
 http://quotes.money.163.com/service/lrb_600585.html?type=year 
 http://quotes.money.163.com/service/xjllb_600585.html?type=year


In [4]:
# 获取三张财务报表CSV格式原始数据
df1 = pd.read_csv(target1, header=0, encoding="gb18030")
df2 = pd.read_csv(target2, header=0, encoding="gb18030")
df3 = pd.read_csv(target3, header=0, encoding="gb18030")

In [5]:
# 报表行列转置，方便进行矢量运算
bs = df1.transpose()
pl = pd.DataFrame(df2.values.T, columns=df2['报告日期'], index=df2.columns)
cf = pd.DataFrame(df3.values.T, columns=df3[' 报告日期'], index=df2.columns)

In [6]:
bs.drop('报告日期', inplace=True)    # 删除第一行
bs.drop(bs.tail(1).index, inplace=True)    # 删除最后一行
pl.drop('报告日期', inplace=True)    # 删除第一行
pl.drop(pl.tail(1).index, inplace=True)    # 删除最后一行
cf.drop('报告日期', inplace=True)    # 删除第一行
cf.drop(cf.tail(1).index, inplace=True)    # 删除最后一行

In [7]:
# 网易提供的CSV文件是字符型数据，无法直接进行算术元算。将字符型数据转换成数值型
headers_bs = bs.columns
headers_pl = pl.columns
headers_cf = cf.columns

for head in headers_bs:
    bs[head] = pd.to_numeric(bs[head], errors='coerce').fillna(0) # 不符合转换条件的设为‘Nan’，再用0替换

for head in headers_pl:
    pl[head] = pd.to_numeric(pl[head], errors='coerce').fillna(0)

for head in headers_cf:
    cf[head] = pd.to_numeric(cf[head], errors='coerce').fillna(0)

In [8]:
# 整理资产负债表
bs_sum = pd.DataFrame()    # 利用column的数字标识进行分类汇总

bs_sum['operat_current'] = bs[0]+bs[6]+bs[7]+bs[8]+bs[12]+bs[13]+bs[14]+bs[15]+bs[17]+bs[20]+bs[21]+bs[22]+bs[23]+bs[19]
bs_sum['operat_non_curr'] = bs[29]+bs[28]+bs[36]+bs[37]+bs[38]+bs[39]+bs[40]+bs[43]+bs[44]+bs[45]+bs[46]+bs[42]+bs[49]+bs[48]
bs_sum['operat_liability'] = bs[57]+bs[60]+bs[59]+bs[64]+bs[63]+bs[67]+bs[66]+bs[70]+bs[71]+bs[72]+bs[69]+bs[78]+bs[79]+bs[80]+bs[77]+bs[82]+bs[87]+bs[91]+bs[90]
bs_sum['financial_assets'] = bs[2]+bs[3]+bs[4]+bs[5]+bs[1]+bs[10]+bs[11]+bs[9]+bs[16]+bs[18]+bs[26]+bs[27]+bs[25]+bs[31]+bs[30]+bs[41]+bs[47]
bs_sum['financial_liability'] = bs[53]+bs[54]+bs[55]+bs[56]+bs[52]+bs[58]+bs[62]+bs[61]+bs[65]+bs[68]+bs[74]+bs[75]+bs[76]+bs[73]+bs[81]+bs[85]+bs[84]+bs[89]+bs[88]
bs_sum['equity'] = bs[95]+bs[94]+bs[98]+bs[97]+bs[99]+bs[101]+bs[103]+bs[105]
# 计算资本支出
bs_sum['operat_net_assets'] = bs_sum['operat_current'] + bs_sum['operat_non_curr'] - bs_sum['operat_liability']
bs_sum['capital_expense'] = bs_sum['operat_net_assets'] - bs_sum['operat_net_assets'].shift(-1)

bs_sum = bs_sum.head(5)   # 只取过去5年数据

total_shares = bs[94]    # 从资产负债表的‘实收资本’中获取流通股数量。疑问:数据中标明是金额万元，但核对后确认是流通股数量

In [9]:
bs_sum.style.format("{:,}")
#bs_sum    # 输出整理完后的资产负债表

,operat_current,operat_non_curr,operat_liability,financial_assets,financial_liability,equity,operat_net_assets,capital_expense
2021-12-31,"8,709,450.0","10,288,618.0","2,252,210.0","3,233,668.0","1,251,286.0","19,190,169.0","16,745,858.0","1,843,994.0"
2020-12-31,"7,447,513.0","8,743,911.0","1,289,560.0","3,356,807.0","1,098,629.0","16,819,758.0","14,901,864.0","2,003,078.0"
2019-12-31,"6,801,131.0","8,120,152.0","2,022,497.0","2,582,363.0","1,224,511.0","14,217,694.0","12,898,786.0","1,241,249.0"
2018-12-31,"5,853,693.0","7,717,077.0","1,913,233.0","1,351,835.0","1,068,405.0","11,630,762.0","11,657,537.0","2,196,100.0"
2017-12-31,"3,466,429.0","7,569,186.0","1,574,178.0","1,178,644.0","1,443,670.0","9,167,292.0","9,461,437.0","754,374.0"


In [11]:
# 整理利润表
pl_sum = pd.DataFrame()

pl_sum['total_revenu'] = pl['营业总收入(万元)']
pl_sum['growth_rate'] = (pl['营业总收入(万元)'] - pl['营业总收入(万元)'].shift(-1)) / pl['营业总收入(万元)'].shift(-1)     # 计算历史每年销售增长率
pl_sum['intre_expen'] = pl['利息支出(万元)']
pl_sum['total_profit'] = pl['利润总额(万元)']
pl_sum['net_profit'] = pl['净利润(万元)']
pl_sum['income_tax'] = pl['所得税费用(万元)']
pl_sum['tax_rate'] = pl['所得税费用(万元)'] / pl['利润总额(万元)']    # 计算企业所得税率

#pl_sum.drop('报告日期', inplace=True)    # 删除第一行
pl_sum = pl_sum.head(5)    # 提取历史5年数据

In [12]:
pd.options.display.float_format = '{:,.2f}'.format    # 设定输出格式
#pl_sum.style.format({'growth_rate':"{:.2}", 'tax_rate':'{:.2}'})
pl_sum     # 输出整理完后的损益表

,total_revenu,growth_rate,intre_expen,total_profit,net_profit,income_tax,tax_rate
2021-12-31,16795266,-0.05,0.00,4411620,3416584,995036,0.23
2020-12-31,17624268,0.12,0.00,4710792,3637018,1073774,0.23
2019-12-31,15703033,0.22,0.00,4455685,3435201,1020484,0.23
2018-12-31,12840263,0.70,0.00,3962920,3063601,899318,0.23
2017-12-31,7531082,0.35,0.00,2122876,1642873,480002,0.23


In [14]:
# 整理现金流量表
cf_sum = pd.DataFrame()
# 提取和资本支出相关项目数据
cf_sum['depr_fixed'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']    # 注意column的标题有一个前置空格
cf_sum['intang_amort'] = cf[' 无形资产摊销(万元)']
cf_sum['deferred'] = cf[' 长期待摊费用摊销(万元)']
# cf_sum['depreciation'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']+cf[' 无形资产摊销(万元)']+cf[' 长期待摊费用摊销(万元)']

#cf_sum.drop('报告日期', inplace=True)    # 删除第一行
cf_sum = cf_sum.head(5)

In [15]:
cf_sum    # 输出整理完的现金流量表

,depr_fixed,intang_amort,deferred
2021-12-31,535191,60485,0.00
2020-12-31,498444,44512,0.00
2019-12-31,493163,33934,0.00
2018-12-31,464730,30611,0.00
2017-12-31,459028,24290,0.00


In [16]:
# 汇总历史估值数据
valuation = pd.DataFrame()

valuation['profit_after_tax'] = pl_sum['net_profit'] + pl_sum['intre_expen'] * (1 - pl_sum['tax_rate'] )
valuation['depreciation'] = cf_sum['depr_fixed'] + cf_sum['intang_amort'] + cf_sum['deferred']
valuation['capital_expense'] = bs_sum['capital_expense']
valuation['net_cash_flow'] = valuation['profit_after_tax'] + valuation['depreciation'] - valuation['capital_expense']

In [17]:
valuation

,profit_after_tax,depreciation,capital_expense,net_cash_flow
2021-12-31,"3,416,584.00","595,676.00","1,843,994.00","2,168,266.00"
2020-12-31,"3,637,018.00","542,956.00","2,003,078.00","2,176,896.00"
2019-12-31,"3,435,201.00","527,097.00","1,241,249.00","2,721,049.00"
2018-12-31,"3,063,601.00","495,341.00","2,196,100.00","1,362,842.00"
2017-12-31,"1,642,873.00","483,318.00","754,374.00","1,371,817.00"


In [18]:
discount_rate = 0.12    # 折现率，仅供参考
continue_rate = 0.05    # 永续经营增长率，通常固定为0.05

# 估算收入的增长率
growth_rate_max = 1+ pl_sum['growth_rate'].max()    # 最大
growth_rate_mean = 1 + pl_sum['growth_rate'].mean()    # 平均
growth_rate_min = 1 + pl_sum['growth_rate'].min()    # 最小
# growth_rate3 = np.percentile(pl_sum['growth_rate'], 75)    # 75分位

growth_rate = growth_rate_min    # 增长率的选择可以调节
growth_rate_continue = 1.05


# 未来5年的增长比例，第6个是永续经营的比例
a = np.logspace(1,5,5, base=growth_rate)
growth_rates = np.append(a, a[4] * growth_rate_continue)
growth_rates


array([0.95296247, 0.90813747, 0.86542093, 0.82471367, 0.78592118,
       0.82521724])

In [19]:
# 预测未来收入，只和增长率相关
revenu_forecast = pl_sum.iloc[0, 0] * growth_rates
revenu_forecast

array([16005258.20480919, 15252410.42342597, 14534974.7406594 ,
       13851285.45892832, 13199755.23094821, 13859742.99249562])

In [20]:
# 预测未来净利润， 只和增长率相关
profit_forecast = valuation.iloc[0,0] * growth_rates
profit_forecast

array([3255876.33434444, 3102727.96001626, 2956783.30663778,
       2817703.52898294, 2685165.72026748, 2819424.00628086])

In [21]:
# 预测未来5年资本支出
bs_rate = bs_sum.sum() / pl_sum['total_revenu'].sum()    # 各资本项占销售收入的历史比率
bs_rate

operat_current        0.46
operat_non_curr       0.60
operat_liability      0.13
financial_assets      0.17
financial_liability   0.09
equity                1.01
operat_net_assets     0.93
capital_expense       0.11
dtype: float64

In [22]:
bs_forecast0 = bs_rate.head(5) * revenu_forecast[0]    # 基于未来的销售收入预测第一年bs项，只提取前5项

# b = np.logspace(0,4,5, base=growth_rate_max)
# np.append(b, b[4] * 1.05)

In [23]:
#np.expand_dims(b, 0).repeat(8, axis=0)
#np.expand_dims(bs_forecast.T, 0).repeat(5, axis=1)
bs_last = bs_sum.iloc[0].head(5)

In [24]:
# 分步计算第二至五年和第六个永续经营的资本支出项
bs_forecast1 = bs_forecast0 * growth_rate
bs_forecast2 = bs_forecast1 * growth_rate
bs_forecast3 = bs_forecast2 * growth_rate
bs_forecast4 = bs_forecast3 * growth_rate
bs_forecast5 = bs_forecast4 * growth_rate_continue

In [25]:
operate = pd.concat([bs_last, bs_forecast0, bs_forecast1, bs_forecast2, bs_forecast3, bs_forecast4, bs_forecast5], axis=1)
operate

,2021-12-31,0,1,2,3,4,5
operat_current,"8,709,450.00","7,328,592.88","6,983,873.99","6,655,369.82","6,342,317.67","6,043,990.73","6,346,190.26"
operat_non_curr,"10,288,618.00","9,635,530.75","9,182,299.20","8,750,386.54","8,338,789.99","7,946,553.92","8,343,881.62"
operat_liability,"2,252,210.00","2,055,134.12","1,958,465.69","1,866,344.30","1,778,556.08","1,694,897.20","1,779,642.06"
financial_assets,"3,233,668.00","2,657,174.29","2,532,187.38","2,413,079.54","2,299,574.25","2,191,407.96","2,300,978.36"
financial_liability,"1,251,286.00","1,381,906.85","1,316,905.37","1,254,961.40","1,195,931.12","1,139,677.47","1,196,661.35"


In [26]:
# 计算当年净负债，供NPV扣减净负债时用
net_liability = operate.iloc[4,0]-operate.iloc[3,0]    # Pandas的iloc函数只看行列的序号，不看行列的标号
net_liability

-1982382.0

In [27]:
# 计算经营性净资产
net_operate = operate.iloc[0]+operate.iloc[1]-operate.iloc[2]

In [28]:
# 计算未来5年和永续经营的资本支出
capex_forecast = net_operate - net_operate.shift(1) 
# 提取前6项
capex_forecast = np.array(capex_forecast)[1:7]

In [29]:
capex_forecast

array([-1836868.48962569,  -701282.01194395,  -668295.43965251,
        -636860.47417975,  -606904.13177642,   614782.37264108])

In [30]:
# 预测5年的折旧
c = np.array(valuation['depreciation'])
c = c[0:3]
depre = c[::-1]

In [31]:
# g = np.full(5, growth_rate_max)    # 用指定增长率构造一个5元数组
# growth_rates = np.append(growth_rates, growth_rate_continue)

for i in np.arange(3,9):    # 构造整数序列
    d = (depre[(i-3)] + depre[(i-2)] + depre[(i-1)]) / 3     # 前三年的平均折旧金额
    d = d * growth_rate
    depre = np.append(depre, d)

depre_forecast = depre[3:9]    # 截取最后6项, 未来5年加永续经营

In [32]:
depre_forecast

array([529125.74182587, 529770.180101  , 525581.64959235, 503315.88778041,
       495117.28034293, 484109.64267143])

In [33]:
# 计算现金流量
forecast_net_cash = profit_forecast + depre_forecast - capex_forecast
ca1 = forecast_net_cash[0:5]    # 只计算今后5年
ca2 = np.concatenate(([0], ca1))    # 当年的现金流要设为0，适应numpy的npv函数设定
contin = forecast_net_cash[5] / (discount_rate - continue_rate)
ca = np.concatenate((ca2, [contin]))    # 添加永续经营现金流

In [34]:
# 计算净现值NPV，此处有瑕疵，永续经营现金流contin所对应的折现系数应采用第五年的值，为方便直接放到npv函数里取到的是第六年的值，使最终估值略偏小！！！
net_present_value = np.npv(discount_rate, ca)    # 利用numpy的npv函数计算净现值
estimated_stock_value = (net_present_value - net_liability) / total_shares    # 净现值扣除当前净负债

In [35]:
# 输出估值
estimated_stock_value[0]    # 最终估值

70.83097535625423

#### 2.财务指标



In [71]:
# 应付-应受 ： 应付票据+应付账款+预收账款 - 应收票据+应收账款+预付账款。无偿占用供销两头的资金大于0
occupied = (bs[58] + bs[59] + bs[60]) - (bs[5] + bs[6] + bs[7])


In [47]:
# 应收占比: 小于3%，大于10%要淘汰。 产品销售难易度，产品竞争力
ar_rate = bs[6] / bs[51]

In [48]:
# 固定资产占比： 不大于40%，固定资产+在建工程+工程物资。 维持竞争力的成本
fixed_rate = (bs[36] + bs[37] + bs[38]) / bs[51]

In [72]:
# 投资类资产占比： 企业专注度。可供出售金融资产+长期股权投资+其他长期投资+投资性房地产
investment = (bs[26] + bs[29] + bs[30] + bs[31]) / bs[51]

In [49]:
# 存货占比: 小于5%
inventory_rate = bs[19] / bs[51]

In [50]:
# 商誉占比: 小于10%
goodwill_rate = bs[45] / bs[51]

In [74]:
# 商品服务收入现金 / 营业收入: 大于110%
cf[' 销售商品、提供劳务收到的现金(万元)'] / pl['营业总收入(万元)']

2021-12-31   1.19
2020-12-31   1.21
2019-12-31   1.25
2018-12-31   1.28
2017-12-31   1.23
2016-12-31   1.24
2015-12-31   1.27
2014-12-31   1.34
2013-12-31   1.28
2012-12-31   1.32
2011-12-31   1.18
2010-12-31   1.17
2009-12-31   1.32
2008-12-31   1.25
2007-12-31   1.14
2006-12-31   1.09
dtype: float64

In [52]:
# 期间费率： 小于40%，成本管控力
sga = (pl['销售费用(万元)'] + pl['管理费用(万元)'] + pl['研发费用(万元)'] + pl['财务费用(万元)']) / pl['营业收入(万元)']    

In [53]:
# ROE: 归属于母公司所有者的净利润 / 归属于母公司股东权益合计
roe = pl['归属于母公司所有者的净利润(万元)'] / bs[104]

In [75]:
# 经营现金流净额： 造血能力
(cf[' 经营活动产生的现金流量净额(万元)'] - cf[' 经营活动产生的现金流量净额(万元)'].shift(-1)) / cf[' 经营活动产生的现金流量净额(万元)'].shift(-1)

2021-12-31   -0.025769
2020-12-31   -0.145833
2019-12-31    0.129766
2018-12-31    1.076767
2017-12-31    0.315705
2016-12-31    0.331906
2015-12-31   -0.438773
2014-12-31    0.161590
2013-12-31    0.320621
2012-12-31    0.096917
2011-12-31    0.745635
2010-12-31   -0.144969
2009-12-31    0.334762
2008-12-31    0.973303
2007-12-31   -0.125763
2006-12-31         NaN
Name:  经营活动产生的现金流量净额(万元), dtype: float64

In [74]:
# 固定资产支付现金占比: 未来成长的能力
cf[' 购建固定资产、无形资产和其他长期资产所支付的现金(万元)'] / cf[' 经营活动产生的现金流量净额(万元)']

2021-12-31    0.448439
2020-12-31    0.286523
2019-12-31    0.217833
2018-12-31    0.131688
2017-12-31    0.210570
2016-12-31    0.377479
2015-12-31    0.521527
2014-12-31    0.395001
2013-12-31    0.497519
2012-12-31    0.592170
2011-12-31    0.786337
2010-12-31    1.541829
2009-12-31    1.138404
2008-12-31    0.978378
2007-12-31    2.092401
2006-12-31    1.144369
dtype: float64

In [ ]:
eps = pl['基本每股收益']

In [77]:
cf[[' 投资活动产生的现金流量净额(万元)',' 筹资活动产生的现金流量净额(万元)']]

报告日期,投资活动产生的现金流量净额(万元),筹资活动产生的现金流量净额(万元)
2021-12-31,-2166696.0,-1160447.0
2020-12-31,-2677288.0,-1326202.0
2019-12-31,-2068885.0,-791190.0
2018-12-31,-2566970.0,-1098000.0
2017-12-31,-520265.0,-749961.0
2016-12-31,-455225.0,-715095.0
2015-12-31,-1271927.0,-539532.0
2014-12-31,-485111.0,-678568.0
2013-12-31,-1247633.0,-428937.0
2012-12-31,-846966.0,-267092.0


#### 3. 历史成交

In [36]:
# 设置最近股价的日期
import datetime as dt 
endday = dt.datetime.today()    # 当天日期
if endday.weekday() == 6:    # 当天是否周日
    end = dt.datetime(endday.year, endday.month, endday.day - 2)
if endday.weekday() == 5:    # 当天是否周六
    end = dt.datetime(endday.year, endday.month, endday.day - 1)    # 把当日设为周五

start = end - dt.timedelta(3,0,0)    # 开始日期往前到推3天
start = start.strftime('%Y%m%d')    # 把日期转换为字符串
end = end.strftime('%Y%m%d')
print(end)

20220812


In [37]:
# 股票代码, 沪市加0，深市加1
if stock[0:1] == '6':
    code = '0' + stock
elif stock[0:1] == '0':
    code = '1' + stock
elif stock[0:1] == '3':
    code = '1' + stock

print(code)

0600585


In [38]:
# 基于网易构造历史成交数据源URL

type = "chddata.html?code="
start = "&start=" + start
end = "&end=" + end

target9 = server + path + type + code + start + end

print(target9)

http://quotes.money.163.com/service/chddata.html?code=0600585&start=20220809&end=20220812


In [39]:
# 获取成交历史原始数据
df9 = pd.read_csv(target9, header=0, encoding="gb18030")


In [40]:
price = df9.loc[0, '收盘价']
price

32.88